## Task 1: Create a cloud client

In [20]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /home/bstollnitz/git/config.json


## Task 2: Register the training and test data

In [21]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

TRAIN_DATA_NAME = 'hospital_train_parquet'
TEST_DATA_NAME = 'hospital_test_parquet'

training_data = Data(
    name=TRAIN_DATA_NAME,
    path='data/training_data.parquet',
    type=AssetTypes.URI_FILE,
    description='RAI hospital train data'
)
tr_data = ml_client.data.create_or_update(training_data)

test_data = Data(
    name=TEST_DATA_NAME,
    path='data/testing_data.parquet',
    type=AssetTypes.URI_FILE,
    description='RAI hospital test data'
)
ts_data = ml_client.data.create_or_update(test_data)

Uploading testing_data.parquet (< 1 MB): 100%|██████████| 23.7k/23.7k [00:00<00:00, 434kB/s]




## Task 3: Create a compute cluster

In [22]:
from azure.ai.ml.entities import AmlCompute

COMPUTE_NAME = 'trainingcompute'

my_compute = AmlCompute(
    name=COMPUTE_NAME,
    size='Standard_DS2_v2',
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=3600
)
ml_client.compute.begin_create_or_update(my_compute)

## Task 4: Create the job

In [23]:
from azure.ai.ml import command, Input, Output

TARGET_COLUMN_NAME = 'readmit_status'

# Create the job.
job = command(
    description='Trains hospital readmission model',
    experiment_name='hospital_readmission',
    compute=COMPUTE_NAME,
    inputs=dict(training_data=Input(type='uri_file', path=f'{TRAIN_DATA_NAME}@latest'), 
                target_column_name=TARGET_COLUMN_NAME),
    outputs=dict(model_output=Output(type=AssetTypes.MLFLOW_MODEL)),
    code='src/',
    environment='azureml://registries/azureml/environments/AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu/versions/4',
    command='python train.py ' + 
            '--training_data ${{inputs.training_data}} ' +
            '--target_column_name ${{inputs.target_column_name}} ' +
            '--model_output ${{outputs.model_output}}'
)
job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(job.name)

RunId: sincere_ice_8p2vf05vk3
Web View: https://ml.azure.com/runs/sincere_ice_8p2vf05vk3?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz

Execution Summary
RunId: sincere_ice_8p2vf05vk3
Web View: https://ml.azure.com/runs/sincere_ice_8p2vf05vk3?wsid=/subscriptions/92a04f75-fa76-4b98-bc2e-ac45670d4044/resourcegroups/rg-bstollnitz/workspaces/mlws-bstollnitz



## Task 5: Register the model

In [24]:
from azure.ai.ml.entities import Model

MODEL_NAME = 'hospital_readmission_model'

# Register the model.
model_path = f"azureml://jobs/{job.name}/outputs/model_output"
model = Model(name=MODEL_NAME,
                path=model_path,
                type=AssetTypes.MLFLOW_MODEL)
registered_model = ml_client.models.create_or_update(model)

## Task 6: Deploy the model

In [25]:
from azure.ai.ml.entities import ManagedOnlineDeployment, ManagedOnlineEndpoint 

ENDPOINT_NAME = 'hospital-readmission-endpoint'
DEPLOYMENT_NAME = 'blue'

# Create the managed online endpoint.
endpoint = ManagedOnlineEndpoint(
    name=ENDPOINT_NAME,
    auth_mode='key',
)
registered_endpoint = ml_client.online_endpoints.begin_create_or_update(
    endpoint)

# Create the managed online deployment.
deployment = ManagedOnlineDeployment(name=DEPLOYMENT_NAME,
                                        endpoint_name=ENDPOINT_NAME,
                                        model=registered_model,
                                        instance_type='Standard_DS4_v2',
                                        instance_count=1)
ml_client.online_deployments.begin_create_or_update(deployment)

# Set deployment traffic to 100%.
registered_endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(
    registered_endpoint)

Check: endpoint hospital-readmission-endpoint exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored
Creating/updating online deployment blue 

.......

## Task 7: Invoke the endpoint

In [ ]:
TEST_DATA_PATH="test_data.json"

# Invoke the endpoint.
result = ml_client.online_endpoints.invoke(endpoint_name=ENDPOINT_NAME, request_file=TEST_DATA_PATH)
print(result)

["not readmitted"]
